<a href="https://colab.research.google.com/github/megcp/Research-project-sentiment-driven-stock-market-prediction/blob/main/RP3_Sentiment_Analysis_of_headlines_using_FinBERT_tone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install torch

In [1]:
import pandas as pd
import numpy as np
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import AutoModelForSequenceClassification, AutoTokenizer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#load clean Dataset
df = pd.read_csv('/content/drive/My Drive/Research Project/ready_apple_articles_2022_2024.csv', index_col= False)
pd.set_option('max_colwidth', None)
df

,Title,Time Published
0,"GOOGL Boosts Podcast Efforts: Should SPOT, AMZN & AAPL Worry?",2022-03-07 15:03:00
1,What to Expect From Apple's First Big Event of The Year,2022-03-07 17:05:00
2,Apple TV+ Enters Comcast Xfinity Platform: Is iPhone Maker's Streaming Push Turning Into A Shove?,2022-03-08 03:16:11
3,Apple Event 2022: New Low-End iPhone Expected to Have 5G Service,2022-03-08 14:33:00
4,What To Expect From Apple's Spring Product Launch Event,2022-03-08 16:15:00
...,...,...
5041,Apple's Upcoming Back To School Promotion Reportedly Features Gift Cards For Mac Buyers - Apple,2024-06-05 05:12:38
5042,"Siri's AI Awakening? iOS 18 Promises iPhone Upgrade Apple Users Have Been Craving: What We Know So Far - NVIDIA , Apple",2024-06-05 09:34:08
5043,"Market Clubhouse Morning Memo - June 5th, 2024 - Invesco QQQ Trust, Series 1",2024-06-05 14:23:29
5044,Does Apple's AI Focus Make it a Buy Ahead of WWDC?,2024-06-05 19:00:00


In [ ]:
# checking shape
df.shape

(5046, 2)

In [ ]:
# check for duplicates
duplicates =df.duplicated()
print(duplicates.sum())

11


In [ ]:
# remove duplicates
df = df.drop_duplicates()
#check for duplicates again- they have all been removed
duplicates =df.duplicated()
print(duplicates.sum())

0


In [ ]:
# remove double spaces
df['Title'] = df['Title'].replace(r'\s+', ' ', regex=True)

<ipython-input-16-970e3f720365>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Title'] = df['Title'].replace(r'\s+', ' ', regex=True)


In [ ]:
df.shape

(5035, 2)

In [ ]:
df.head(2)

,Title,Time Published
0,"GOOGL Boosts Podcast Efforts: Should SPOT, AMZN & AAPL Worry?",2022-03-07 15:03:00
1,What to Expect From Apple's First Big Event of The Year,2022-03-07 17:05:00


In [ ]:
# seperate the headlines form the rest of the dataframe by putting them in to a list
X = df['Title']

In [ ]:
X

0                                                                                     GOOGL Boosts Podcast Efforts: Should SPOT, AMZN & AAPL Worry?
1                                                                                           What to Expect From Apple's First Big Event of The Year
2                                                 Apple TV+ Enters Comcast Xfinity Platform: Is iPhone Maker's Streaming Push Turning Into A Shove?
3                                                                                  Apple Event 2022: New Low-End iPhone Expected to Have 5G Service
4                                                                                           What To Expect From Apple's Spring Product Launch Event
                                                                           ...                                                                     
5030                                               Apple's Upcoming Back To School Promotion Reportedly Features

In [ ]:
# check the lngth of X is the same as before
len(X)

5035

In [ ]:
# Load finbert-tone model and initialise tokeniser
model = AutoModelForSequenceClassification.from_pretrained("yiyanghkust/finbert-tone")
tokenizer = AutoTokenizer.from_pretrained("yiyanghkust/finbert-tone")


config.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

In [ ]:
# create labels for the numeric outputs
labels = {0:'neutral', 1:'positive',2:'negative'}

## sentiment anlysis of the title extracting both the individual probalitites and the overal sentiment

In [ ]:
# code adapted from (Konstantinos Poulinakis, 2022 and Shinde, 2021)
texts = []
sentiment_labels = []
sentiment_probs = []

for x in X:
    # tokenize the headline
    inputs = tokenizer(x, return_tensors="pt", padding=True, truncation=True, max_length=512)
    # pass the input to the FinBERT model
    outputs = model(**inputs)[0]
    # apply softmax to get probabilities
    probs = torch.nn.functional.softmax(outputs, dim=1).detach().cpu().numpy().flatten()
    # find the label with the highest probability
    sentiment_label = labels[np.argmax(probs)]
    texts.append(x)
    sentiment_labels.append(sentiment_label)
    sentiment_probs.append(probs)

df1 = pd.DataFrame({
    'Title': texts,
    'Sentiment_title': sentiment_labels,
    'Neutral_Prob_title': [prob[0] for prob in sentiment_probs],
    'Positive_Prob_title': [prob[1] for prob in sentiment_probs],
    'Negative_Prob_title': [prob[2] for prob in sentiment_probs]
})

### .detach() = Returns a new Tensor, detached from the current graph
### .cpu() = returns in cpu memory need to convert to a numpy
### .numpy() and convert to numpy.
### .flattern() to return an flatterned list of probalitlies

In [ ]:
# view output
df1

,Title,Sentiment_title,Neutral_Prob_title,Positive_Prob_title,Negative_Prob_title
0,"GOOGL Boosts Podcast Efforts: Should SPOT, AMZN & AAPL Worry?",positive,0.000001,9.999988e-01,1.425253e-07
1,What to Expect From Apple's First Big Event of The Year,neutral,0.999801,7.346754e-05,1.254924e-04
2,Apple TV+ Enters Comcast Xfinity Platform: Is iPhone Maker's Streaming Push Turning Into A Shove?,neutral,0.997163,5.182165e-04,2.319331e-03
3,Apple Event 2022: New Low-End iPhone Expected to Have 5G Service,neutral,0.999533,4.479716e-04,1.881107e-05
4,What To Expect From Apple's Spring Product Launch Event,neutral,0.997772,2.058819e-04,2.022169e-03
...,...,...,...,...,...
5030,Apple's Upcoming Back To School Promotion Reportedly Features Gift Cards For Mac Buyers - Apple,neutral,0.999988,7.140618e-06,5.135410e-06
5031,"Siri's AI Awakening? iOS 18 Promises iPhone Upgrade Apple Users Have Been Craving: What We Know So Far - NVIDIA , Apple",negative,0.049752,1.320929e-02,9.370385e-01
5032,"Market Clubhouse Morning Memo - June 5th, 2024 - Invesco QQQ Trust, Series 1",neutral,0.999995,8.105184e-07,3.908215e-06
5033,Does Apple's AI Focus Make it a Buy Ahead of WWDC?,neutral,0.881488,9.080873e-02,2.770283e-02


In [ ]:
# making time published into its own data frame to join to the sentiment values df
time_published = df[['Time Published']]

In [ ]:
# check to see if they are in correct order - they are
time_published. head(2)

,Time Published
0,2022-03-07 15:03:00
1,2022-03-07 17:05:00


In [ ]:
# join the sentiment dataframe with the published time dataframe
final_df = pd.concat([time_published, df1], axis=1)

In [ ]:
#view final dataset
final_df

,Time Published,Title,Sentiment_title,Neutral_Prob_title,Positive_Prob_title,Negative_Prob_title
0,2022-03-07 15:03:00,"GOOGL Boosts Podcast Efforts: Should SPOT, AMZN & AAPL Worry?",positive,0.000001,9.999988e-01,1.425253e-07
1,2022-03-07 17:05:00,What to Expect From Apple's First Big Event of The Year,neutral,0.999801,7.346754e-05,1.254924e-04
2,2022-03-08 03:16:11,Apple TV+ Enters Comcast Xfinity Platform: Is iPhone Maker's Streaming Push Turning Into A Shove?,neutral,0.997163,5.182165e-04,2.319331e-03
3,2022-03-08 14:33:00,Apple Event 2022: New Low-End iPhone Expected to Have 5G Service,neutral,0.999533,4.479716e-04,1.881107e-05
4,2022-03-08 16:15:00,What To Expect From Apple's Spring Product Launch Event,neutral,0.997772,2.058819e-04,2.022169e-03
...,...,...,...,...,...,...
5030,2024-06-05 05:12:38,Apple's Upcoming Back To School Promotion Reportedly Features Gift Cards For Mac Buyers - Apple,neutral,0.999988,7.140618e-06,5.135410e-06
5031,2024-06-05 09:34:08,"Siri's AI Awakening? iOS 18 Promises iPhone Upgrade Apple Users Have Been Craving: What We Know So Far - NVIDIA , Apple",negative,0.049752,1.320929e-02,9.370385e-01
5032,2024-06-05 14:23:29,"Market Clubhouse Morning Memo - June 5th, 2024 - Invesco QQQ Trust, Series 1",neutral,0.999995,8.105184e-07,3.908215e-06
5033,2024-06-05 19:00:00,Does Apple's AI Focus Make it a Buy Ahead of WWDC?,neutral,0.881488,9.080873e-02,2.770283e-02


In [ ]:
# view number headline beloging to each sentiment class
final_df['Sentiment_title'].value_counts()

Sentiment_title
neutral     2985
positive    1074
negative     976
Name: count, dtype: int64

In [ ]:
#save the final dataframe
final_df.to_csv('/content/drive/My Drive/Research Project/apple_sentiment_probabilites_2022_2024.csv', index= False)

reference:

Konstantinos Poulinakis. (2022, April 18). Stock News Sentiment analysis with Transformers | CodeX. Medium; CodeX. https://medium.com/codex/stocks-news-sentiment-analysis-with-deep-learning-transformers-and-machine-learning-cdcdb827fc06


Shinde, R. (2021, October 10). Financial News Sentiment Analysis using FinBERT - Raviraj Shinde - Medium. Medium; Medium. https://medium.com/@ravirajshinde2000/financial-news-sentiment-analysis-using-finbert-25afcc95e65f
